In [1]:
import urllib
import requests
import os
import itertools
import math
from copy import deepcopy as mcopy
from music21 import *

In [2]:
# Return a generator that iterates over all permutations of a list
# http://stackoverflow.com/questions/2710713/algorithm-to-generate-all-possible-permutations-of-a-list
def permute(xs, low=0):
    if low + 1 >= len(xs):
        yield xs
    else:
        for p in permute(xs, low + 1):
            yield p        
        for i in range(low + 1, len(xs)):        
            xs[low], xs[i] = xs[i], xs[low]
            for p in permute(xs, low + 1):
                yield p        
            xs[low], xs[i] = xs[i], xs[low]

def nth_permutation_indices(i, m):
    available_positions = list(range(m))
    indices = [0 for r in range(m)]
    res     = i
    for row_index in range(m-1):
        # How many posibilities there are in next rows?
        n1f      = math.factorial(m - row_index -1)
        # What is the index of this row?
        res      = math.floor(i / n1f)
        # The index may have already been used in a previous row.
        # take one from our list of indexes
        position = available_positions[res]
        del available_positions[res]

        indices[row_index] = position
        # our next iteration through the loop will be constrained by 
        i = i - (res * n1f)
        
    indices[m-1] = available_positions[0]
    return indices


/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
# Initialize Music21 and 

defaults.author = ''
defaults.title = ''

def send_m21_object(obj, scoreName):
    xml = musicxml.m21ToString.fromMusic21Object(obj)
    headers = {'Content-Type': 'text/xml'}
    url = urllib.parse.urljoin('http://ks-images:3000/score/', scoreName)
    return requests.post(url, 
                        data=xml,
                        headers=headers,
                        timeout=5)
def write_midi_file(a_stream, filename='default.mid'):
    # streanToMidiFile is buggy for non-flat streams
    mf = midi.translate.streamToMidiFile(a_stream.flat)
    mf.open(filename, 'wb')
    mf.write()
    mf.close()

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
def append_octave_up(stream):
    """Append a copy of the stream an octave up (in place)"""
    for n in stream.notes:
        stream.append(n.transpose(12))
    return stream

def all_but_last(a_stream):
    """Return a new stream that includes all but the last note.
    Discard any non-note items in the stream.
    """
    Class = a_stream.__class__
    result = Class()
    notes = list(mcopy(a_stream).notesAndRests)
    if len(notes) <= 1:
        return result
    result.append(notes[:-1])
    return result

def all_but_first_and_last(a_stream):
    """Return a new stream of the same type that includes all but
    the first and last rests/notes"""
    Class = a_stream.__class__
    result = Class()
    notes = list(mcopy(a_stream).notesAndRests)
    if len(notes) <= 2:
        return result
    result.append(notes[1:-1])
    return result

def reverse_notes(a_stream):
    """Return a new stream with the notes and rests reversed."""
    Class = a_stream.__class__
    result = Class()
    notes = list(mcopy(a_stream).notesAndRests)
    notes.reverse()
    result.append(notes)
    return result

def all_but_first(a_stream):
    """"""
    return reverse_notes(all_but_last(reverse_notes(a_stream)))

def arp_forward_rev_x_times(a_stream, x):
    """Arpeggiate a stream of notes x times. Playback the stream
    in reverse order, without playing the last note in the
    original stream twice. Go through and back x times."""
    
    Class = a_stream.__class__
    result = Class()

    # All but last reversed
    abflr = reverse_notes(all_but_first_and_last(a_stream))
    result.append(a_stream)
    result.append(abflr)

    appendage = mcopy(result).flat
    result.repeatAppend(appendage, x - 1)
    return result.flat


In [5]:
def hashtag_music(note_list):
    s1 = stream.Part()
    noteList = [note.Note(n, type='16th') for n in note_list]
    s1.append(noteList)
    append_octave_up(s1)
    return arp_forward_rev_x_times(s1, 1)

In [11]:
n1 = ['C3', 'D3', 'G3', 'A3']
n1_midi = [note.Note(name).pitch.midi for name in n1]
stream1 = hashtag_music(n1)
stream1.insert(clef.BassClef())
send_m21_object(stream1, 'a')

n2 = ['C3', 'e3', 'G3', 'b3']
n2_midi = [note.Note(name).pitch.midi for name in n2]
 
stream2 = hashtag_music(n2)
stream2.insert(clef.BassClef())
send_m21_object(stream2, 'a')

n3 = ['D3', 'E3', 'A3', 'B3']
n3_midi = [note.Note(name).pitch.midi for name in n3]
stream3 = hashtag_music(n3)
stream3.insert(clef.BassClef())
send_m21_object(stream3, 'a')

<Response [200]>

In [7]:
cycle_me = [n1_midi, n1_midi, n2_midi, n2_midi, n3_midi, n3_midi]
cycle_me = [ns + [n + 12 for n in ns] for ns in cycle_me] # append upper octave

In [34]:
# Randomize and mirror
permutation_stream = stream.Stream()
for notes_midi, order, r in zip(itertools.cycle(cycle_me), permute(list(range(8))), range(200)):
    
    re_ordered_midi = [notes_midi[i] for i in order]

    # ramp arp back down
    reversed_midi = re_ordered_midi[1: -1]
    reversed_midi.reverse()    
    forward_back_midi = re_ordered_midi + reversed_midi
        
    # add 
    permutation_stream.append([note.Note(num, type='16th') for num in forward_back_midi])

In [8]:
# Randomize full forward and back

initial_order = nth_permutation_indices(0, 14)
permutation_stream = stream.Stream()
for notes_midi, order, r in zip(itertools.cycle(cycle_me), permute(initial_order), range(100)):
    
    # ramp arp back down
    reversed_midi = notes_midi[1: -1]
    reversed_midi.reverse()    
    forward_back_midi = notes_midi + reversed_midi
    
    re_ordered_midi = [forward_back_midi[i] for i in order]
        
    # add 
    permutation_stream.append([note.Note(num, type='16th') for num in re_ordered_midi])

In [9]:
write_midi_file(permutation_stream, '/session/default.mid')

In [48]:
write_midi_file(stream3)

In [43]:
send_m21_object(stream1.flat, 'a')

<Response [200]>

In [49]:
three_sections = stream.Stream()
three_sections.append([stream1, stream2, stream3])
write_midi_file(three_sections.flat)